In [1]:
%pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [3]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Santander Kaggle Challenge/Data Preprocessed/preprocessed_train_data.csv")

In [4]:
data.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191_unique,var_192_unique,var_193_unique,var_194_unique,var_195_unique,var_196_unique,var_197_unique,var_198_unique,var_199_unique,target
0,-0.577102,-1.273737,0.451707,-0.833709,0.235571,-0.536430,-0.334926,0.608751,-1.561580,-1.473796,...,2.010112,0.654905,0.514851,2.809460,-0.579091,1.740727,0.871211,0.520763,0.042849,0
1,0.269959,-0.622138,1.190360,-0.688846,0.790975,1.539900,0.244461,-0.003525,0.858974,0.419300,...,0.542411,0.654905,-0.679283,0.162290,-0.883795,1.017514,2.429465,1.001117,0.042849,0
2,-0.681113,-0.276066,0.516988,0.536516,-0.305477,-0.511033,1.769839,-0.564749,-1.561370,-1.307408,...,0.542411,-0.241114,-0.679283,-0.896578,-1.493204,0.294301,0.425995,0.040410,0.042849,0
3,0.125158,-0.129426,-0.667575,0.195355,0.927992,0.410672,0.500633,-0.474201,-1.843910,0.548767,...,-0.436056,-0.241114,0.514851,0.162290,0.335023,1.740727,-1.354866,-0.920298,0.042849,0
4,-0.277303,0.035610,0.817683,-0.077829,0.738607,0.955574,0.613372,0.791544,1.794753,0.090006,...,2.010112,-1.734481,-1.276349,-1.426012,0.639727,1.740727,-0.241828,-0.920298,0.042849,0


In [5]:
print(data.shape)

(200000, 401)


In [6]:
X,y = data.iloc[:,:-1], data.iloc[:,-1]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_dim=X_train.shape[1], activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Für binäre Klassifikation
])



# Definiere Early Stopping and Model Checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_auc', mode='max', save_best_only=True)

# Kompiliere das ModellAUC(name='auc')]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name="auc")])

# Trainiere das Modell
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping,model_checkpoint])

#load best weights
model.load_weights('best_model.h5')

# Evaluierung des Modells auf den Testdaten
loss, auc = model.evaluate(X_test, y_test)
print(f'Maximaler val_auc Score: {auc:.2f}')

# Finde den besten val_auc Score im Verlauf des Trainings
best_val_auc = max(history.history['val_auc'])
print(f'Bester val_auc Score während des Trainings: {best_val_auc:.2f}')

Epoch 1/50
4000/4000 [==============================] - 16s 3ms/step - loss: 0.2496 - auc: 0.8322 - val_loss: 0.2362 - val_auc: 0.8534
Epoch 2/50
  59/4000 [..............................] - ETA: 10s - loss: 0.2208 - auc: 0.8628

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4000/4000 [==============================] - 13s 3ms/step - loss: 0.2230 - auc: 0.8716 - val_loss: 0.2379 - val_auc: 0.8524
Epoch 3/50
4000/4000 [==============================] - 13s 3ms/step - loss: 0.2094 - auc: 0.8885 - val_loss: 0.2437 - val_auc: 0.8470
Epoch 4/50
4000/4000 [==============================] - 12s 3ms/step - loss: 0.1945 - auc: 0.9047 - val_loss: 0.2549 - val_auc: 0.8411
Epoch 5/50
4000/4000 [==============================] - 13s 3ms/step - loss: 0.1794 - auc: 0.9190 - val_loss: 0.2687 - val_auc: 0.8256
Epoch 6/50
4000/4000 [==============================] - 13s 3ms/step - loss: 0.1645 - auc: 0.9322 - val_loss: 0.2899 - val_auc: 0.8126
Epoch 7/50
1250/1250 [==============================] - 2s 2ms/step - loss: 0.2430 - auc: 0.8481
Maximaler val_auc Score: 0.85
Bester val_auc Score während des Trainings: 0.85


In [9]:
model_dropout_batchnorm = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # 50% der Neuronen zufällig ausschalten
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Kompiliere das Modell
optimizer = Adam(learning_rate=0.001)
model_dropout_batchnorm.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name="auc")])

# Definiere Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_dropout_batchnorm.h5', monitor='val_auc', mode='max', save_best_only=True)


# Trainiere das Modell
history = model_dropout_batchnorm.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping,model_checkpoint])

#load best weights
model_dropout_batchnorm.load_weights('best_model_dropout_batchnorm.h5')

# Evaluierung des Modells auf den Testdaten
loss, auc = model_dropout_batchnorm.evaluate(X_test, y_test)
print(f'Maximaler val_auc Score: {auc:.2f}')

# Finde den besten val_auc Score im Verlauf des Trainings
best_val_auc = max(history.history['val_auc'])
print(f'Bester val_auc Score während des Trainings: {best_val_auc:.2f}')

Epoch 1/100
4000/4000 [==============================] - 21s 4ms/step - loss: 0.2798 - auc: 0.7741 - val_loss: 0.2366 - val_auc: 0.8631
Epoch 2/100
  43/4000 [..............................] - ETA: 14s - loss: 0.2498 - auc: 0.8065

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4000/4000 [==============================] - 17s 4ms/step - loss: 0.2459 - auc: 0.8398 - val_loss: 0.2307 - val_auc: 0.8654
Epoch 3/100
4000/4000 [==============================] - 17s 4ms/step - loss: 0.2399 - auc: 0.8496 - val_loss: 0.2309 - val_auc: 0.8666
Epoch 4/100
4000/4000 [==============================] - 17s 4ms/step - loss: 0.2363 - auc: 0.8566 - val_loss: 0.2289 - val_auc: 0.8658
Epoch 5/100
4000/4000 [==============================] - 17s 4ms/step - loss: 0.2342 - auc: 0.8599 - val_loss: 0.2288 - val_auc: 0.8666
Epoch 6/100
4000/4000 [==============================] - 17s 4ms/step - loss: 0.2309 - auc: 0.8645 - val_loss: 0.2289 - val_auc: 0.8663
Epoch 7/100
4000/4000 [==============================] - 17s 4ms/step - loss: 0.2278 - auc: 0.8701 - val_loss: 0.2299 - val_auc: 0.8662
Epoch 8/100
4000/4000 [==============================] - 17s 4ms/step - loss: 0.2252 - auc: 0.8728 - val_loss: 0.2314 - val_auc: 0.8641
Epoch 9/100
4000/4000 [==============================] - 17s

In [10]:
model_simple = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Kompiliere das Modell
optimizer = Adam(learning_rate=0.001)
model_simple.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name="auc")])

# Definiere Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_simple.h5', monitor='val_auc', mode='max', save_best_only=True)

# Trainiere das Modell
history = model_simple.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping,model_checkpoint])

#load best weights
model_simple.load_weights('best_model_simple.h5')

# Evaluierung des Modells auf den Testdaten
loss, auc = model_simple.evaluate(X_test, y_test)
print(f'Maximaler val_auc Score: {auc:.2f}')

# Finde den besten val_auc Score im Verlauf des Trainings
best_val_auc = max(history.history['val_auc'])
print(f'Bester val_auc Score während des Trainings: {best_val_auc:.2f}')

Epoch 1/100
4000/4000 [==============================] - 15s 3ms/step - loss: 0.2530 - auc: 0.8258 - val_loss: 0.2342 - val_auc: 0.8574
Epoch 2/100
  54/4000 [..............................] - ETA: 11s - loss: 0.2205 - auc: 0.8853

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4000/4000 [==============================] - 13s 3ms/step - loss: 0.2261 - auc: 0.8677 - val_loss: 0.2346 - val_auc: 0.8571
Epoch 3/100
4000/4000 [==============================] - 13s 3ms/step - loss: 0.2177 - auc: 0.8791 - val_loss: 0.2381 - val_auc: 0.8524
Epoch 4/100
4000/4000 [==============================] - 13s 3ms/step - loss: 0.2106 - auc: 0.8885 - val_loss: 0.2435 - val_auc: 0.8465
Epoch 5/100
4000/4000 [==============================] - 13s 3ms/step - loss: 0.2032 - auc: 0.8969 - val_loss: 0.2518 - val_auc: 0.8406
Epoch 6/100
4000/4000 [==============================] - 13s 3ms/step - loss: 0.1956 - auc: 0.9054 - val_loss: 0.2559 - val_auc: 0.8330
Epoch 7/100
1250/1250 [==============================] - 2s 2ms/step - loss: 0.2420 - auc: 0.8504
Maximaler val_auc Score: 0.85
Bester val_auc Score während des Trainings: 0.86


In [11]:
model_complex = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Kompiliere das Modell
optimizer = Adam(learning_rate=0.001)
model_complex.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name="auc")])

# Definiere Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_complex.h5', monitor='val_auc', mode='max', save_best_only=True)

# Trainiere das Modell
history = model_complex.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping,model_checkpoint])

#load best weights
model_complex.load_weights('best_model_complex.h5')

# Evaluierung des Modells auf den Testdaten
loss, auc = model_complex.evaluate(X_test, y_test)
print(f'Maximaler val_auc Score: {auc:.2f}')

# Finde den besten val_auc Score im Verlauf des Trainings
best_val_auc = max(history.history['val_auc'])
print(f'Bester val_auc Score während des Trainings: {best_val_auc:.2f}')

Epoch 1/100
4000/4000 [==============================] - 19s 4ms/step - loss: 0.2458 - auc: 0.8377 - val_loss: 0.2349 - val_auc: 0.8612
Epoch 2/100
  44/4000 [..............................] - ETA: 14s - loss: 0.2176 - auc: 0.8810

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4000/4000 [==============================] - 16s 4ms/step - loss: 0.2233 - auc: 0.8702 - val_loss: 0.2365 - val_auc: 0.8594
Epoch 3/100
4000/4000 [==============================] - 16s 4ms/step - loss: 0.2087 - auc: 0.8864 - val_loss: 0.2419 - val_auc: 0.8564
Epoch 4/100
4000/4000 [==============================] - 16s 4ms/step - loss: 0.1859 - auc: 0.9074 - val_loss: 0.2606 - val_auc: 0.8397
Epoch 5/100
4000/4000 [==============================] - 16s 4ms/step - loss: 0.1554 - auc: 0.9360 - val_loss: 0.3027 - val_auc: 0.8097
Epoch 6/100
4000/4000 [==============================] - 16s 4ms/step - loss: 0.1221 - auc: 0.9612 - val_loss: 0.3607 - val_auc: 0.7822
Epoch 7/100
1250/1250 [==============================] - 3s 2ms/step - loss: 0.2416 - auc: 0.8548
Maximaler val_auc Score: 0.85
Bester val_auc Score während des Trainings: 0.86


Finding the best model

In [12]:
from sklearn.metrics import roc_auc_score

models = {
  "model" : model,
  "model_dropout_batchnorm": model_dropout_batchnorm,
  "model_simple": model_simple,
  "model_complex" : model_complex
}

scores = []

for model_name, model in models.items():
    predictions = model.predict(X_test)

    # Berechne den ROC AUC Score mit Wahrscheinlichkeiten
    roc_auc = roc_auc_score(y_test, predictions)
    print(f"ROC AUC Score for model {model_name}: ", roc_auc)
    scores.append(roc_auc)

# Finde das stärkste Modell
strongest_model_name = list(models.keys())[np.argmax(scores)]
strongest_model = models[strongest_model_name]
print(f"Strongest model: {strongest_model_name} with ROC AUC: {max(scores):.2f}")

1250/1250 [==============================] - 2s 1ms/step
ROC AUC Score for model model:  0.8483712599601979
1250/1250 [==============================] - 2s 1ms/step
ROC AUC Score for model model_dropout_batchnorm:  0.8619926241883361
1250/1250 [==============================] - 2s 1ms/step
ROC AUC Score for model model_simple:  0.8506593998415618
1250/1250 [==============================] - 2s 1ms/step
ROC AUC Score for model model_complex:  0.8548326940179601
Strongest model: model_dropout_batchnorm with ROC AUC: 0.86


In [13]:
model_name = 'model_V1' # @param {type:"string"}
strongest_model.save(f"/content/drive/MyDrive/Colab Notebooks/Santander Kaggle Challenge/Models/{model_name}.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
